# Engenharia de Atributos

Importando as dependências

In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Lendo os dados brutos e concatenando-os em um único dataframe

In [159]:
base_path = '../data/raw/' 

dados_2021 = pd.read_csv(base_path + '2021.csv', encoding='latin-1', sep=';')
dados_2022 = pd.read_csv(base_path + '2022.csv', sep=',')
dados_2023 = pd.read_csv(base_path + '2023.csv', encoding='latin-1', sep=';')
dados_2024 = pd.read_csv(base_path + '2024.csv', sep=',')

dados = pd.concat([dados_2021, dados_2022, dados_2023, dados_2024], axis=0, ignore_index=True)

dados.shape

(250889, 30)

Na análise exploratória, foram indentificadas colunas em que haviam dados nulos:

In [160]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]

uop                       219
delegacia                  85
regional                   23
classificacao_acidente      4
dtype: int64

Reconhecendo os dados dessas colunas

In [161]:
dados['uop'].head(5), dados['delegacia'].head(5), dados['regional'].head(5), dados['classificacao_acidente'].head(5)

(0    UOP01-DEL05-SP
 1    UOP01-DEL01-AM
 2    UOP03-DEL04-CE
 3    UOP02-DEL04-RJ
 4    UOP04-DEL05-RJ
 Name: uop, dtype: object,
 0    DEL05-SP
 1    DEL01-AM
 2    DEL04-CE
 3    DEL04-RJ
 4    DEL05-RJ
 Name: delegacia, dtype: object,
 0    SPRF-SP
 1    SPRF-AM
 2    SPRF-CE
 3    SPRF-RJ
 4    SPRF-RJ
 Name: regional, dtype: object,
 0    Com Vítimas Feridas
 1                    NaN
 2    Com Vítimas Feridas
 3    Com Vítimas Feridas
 4    Com Vítimas Feridas
 Name: classificacao_acidente, dtype: object)

Substituindo os dados faltantes pela moda das respectivas colunas mas considerando outras colunas 

In [162]:
valores = {
    'uop': dados['uop'].mode()[0],
    'delegacia': dados['delegacia'].mode()[0],
    'regional': dados['regional'].mode()[0],
    'classificacao_acidente': dados['classificacao_acidente'].mode()[0]
}

valores

{'uop': 'UOP01-DEL01-SC',
 'delegacia': 'DEL01-PR',
 'regional': 'SPRF-MG',
 'classificacao_acidente': 'Com Vítimas Feridas'}

In [ ]:
dados[dados['uop'].isnull()][['uop', 'uf']]


,uop,uf
2613,NaN,ES
5508,NaN,MT
8653,NaN,RO
12393,NaN,SC
12903,NaN,ES
...,...,...
246295,NaN,SP
246675,NaN,PR
249004,NaN,PA
250805,NaN,PA


In [164]:
#dados = dados.fillna(value=valores)

Checando se os valores nulos foram realmente preenchidos. Como nenhuma coluna com valor nulo é impressa, não existem mais valores nulos na base.

In [165]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]


uop                       219
delegacia                  85
regional                   23
classificacao_acidente      4
dtype: int64

In [166]:
dados.head(2)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,331730.0,2021-01-01,sexta-feira,05:30:00,SP,116,453,REGISTRO,Reação tardia ou ineficiente do condutor,Tombamento,...,0,0,0,1,1,"-24,56168114","-47,86752174",SPRF-SP,DEL05-SP,UOP01-DEL05-SP
1,331804.0,2021-01-01,sexta-feira,08:05:00,AM,174,937,MANAUS,Reação tardia ou ineficiente do condutor,Colisão traseira,...,0,5,1,1,5,"-2,508068","-60,036434",SPRF-AM,DEL01-AM,UOP01-DEL01-AM


Tratamento de data/hora, mudando para o formato datetime

In [167]:
dados['datetime'] = pd.to_datetime(dados['data_inversa'] + ' ' + dados['horario'])

# padrão americano
dados['datetime'] = dados['datetime'].dt.strftime('%m-%d-%Y %I:%M %p')

dados = dados.drop(columns=['data_inversa', 'horario'])

dados

,id,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,...,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,datetime
0,331730.0,sexta-feira,SP,116,453,REGISTRO,Reação tardia ou ineficiente do condutor,Tombamento,Com Vítimas Feridas,Amanhecer,...,0,0,1,1,"-24,56168114","-47,86752174",SPRF-SP,DEL05-SP,UOP01-DEL05-SP,01-01-2021 05:30
1,331804.0,sexta-feira,AM,174,937,MANAUS,Reação tardia ou ineficiente do condutor,Colisão traseira,NaN,Pleno dia,...,5,1,1,5,"-2,508068","-60,036434",SPRF-AM,DEL01-AM,UOP01-DEL01-AM,01-01-2021 08:05
2,331815.0,sexta-feira,CE,222,"128,5",ITAPAJE,Velocidade Incompatível,Tombamento,Com Vítimas Feridas,Pleno dia,...,0,11,2,12,"-3,707626","-39,623509",SPRF-CE,DEL04-CE,UOP03-DEL04-CE,01-01-2021 10:10
3,331823.0,sexta-feira,RJ,493,18,MAGE,Ingestão de álcool e/ou substâncias psicoativa...,Colisão lateral,Com Vítimas Feridas,Pleno dia,...,1,1,1,3,"-22,6687122","-43,01862696",SPRF-RJ,DEL04-RJ,UOP02-DEL04-RJ,01-01-2021 12:30
4,331843.0,sexta-feira,RJ,393,252,BARRA DO PIRAI,Condutor Dormindo,Colisão frontal,Com Vítimas Feridas,Pleno dia,...,1,1,2,3,"-22,43422023","-43,77561487",SPRF-RJ,DEL05-RJ,UOP04-DEL05-RJ,01-01-2021 02:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250884,635554.0,sexta-feira,BA,324,615,SALVADOR,Condutor deixou de manter distância do veículo...,Colisão lateral mesmo sentido,Com Vítimas Feridas,Pleno dia,...,1,0,1,2,"-12,87206398","-38,42746403",SPRF-BA,DEL01-BA,UOP01-DEL01-BA,09-27-2024 04:15
250885,635630.0,domingo,RJ,116,108,GUAPIMIRIM,Condutor deixou de manter distância do veículo...,Colisão traseira,Com Vítimas Feridas,Plena Noite,...,1,0,1,2,"-22,535843","-42,99125",SPRF-RJ,DEL04-RJ,UOP01-DEL04-RJ,09-15-2024 08:30
250886,635699.0,terça-feira,AP,210,3,MACAPA,Transitar na calçada,Colisão lateral sentido oposto,Sem Vítimas,Pleno dia,...,3,0,0,2,"0,08723365","-51,08569007",SPRF-AP,DEL01-AP,UOP01-DEL01-AP,08-13-2024 04:22
250887,635887.0,sábado,PR,277,679,MEDIANEIRA,Manobra de mudança de faixa,Capotamento,Com Vítimas Feridas,Plena Noite,...,1,0,1,2,"-25,31758044","-54,15684223",SPRF-PR,DEL05-PR,UOP01-DEL05-PR,09-21-2024 06:50


In [174]:
for index, row in dados.iterrows():
    dados_filtrados = dados[dados['uf'] == row['uf']]
    moda_uop = dados_filtrados['uop'].mode()[0]

KeyboardInterrupt: 